In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype = {'type': np.int8, 'clear_text': str,}
path = 'C:/Users/user/sentiment/clear_binary_twitts.csv' #home

In [3]:
df = pd.read_csv (path, sep=',', dtype=dtype)
#df

In [4]:
df.head()

,type,clear_text
0,1,школот повер сам обществ профилир предмет тип
1,1,так немн похож мальчик равн
2,1,идиотк испуга
3,1,угл сид погиба голод порц взял хот жрат хот
4,1,знач страшилк блин посмотр част создаст ощущен...


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42)

In [6]:
train.head(), test.head()

(        type                                      clear_text
 26253      1                      игра красн бел форм нарост
 120387    -1                            класс погоня возьмут
 192108    -1  лучш сервис расскрутк ваш аккаунт социальн сет
 131248    -1       наверн буд реша задач хим физик вечн пздц
 119281    -1                      буд примерн недел связ обл,
         type                                         clear_text
 215081    -1                                   мыс обидел сдела
 220754    -1                      ааааааааааааааааааа хоч работ
 60463      1    особен вид некотор вакуумн наушник прост держат
 213263    -1  неплох человек жизн работа биз мног знаком вст...
 113055     1  домаааа еха такс пел песн отдыха крича улиц на...)

In [7]:
t1 = test.type[test.type  == 1].count()
t0 = test.type[test.type  == -1].count()
print(t1,t0)

11408 11276


# Модель RandomForest (35) emb. BoW (3500) 0.6942


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [9]:
#_____________________________________________bag_of_words________________________________________________

corpus = pd.concat([train, test], ignore_index=True) 

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 3500)

X = vectorizer.fit_transform(corpus['clear_text'].values.astype('U'))
#print(vectorizer.get_feature_names())
bd = X.toarray()
#print(bd)

In [10]:
bd.shape, train.shape, test.shape

((226834, 3500), (204150, 2), (22684, 2))

In [11]:
forest = RandomForestClassifier(n_estimators = 35) # количество деревьев - 35
forest = forest.fit(bd[0:train.shape[0]], train['type'])

In [12]:
print('RandomForest', accuracy_score(test['type'], forest.predict(bd[train.shape[0]:bd.shape[0]])))

RandomForest 0.6941897372597425


# Модель LogReg  emb. TF-IDF (word,char n-gram)  0.7061

In [13]:
# TF-IDF(word)

tfv_word = TfidfVectorizer(min_df=150,  max_features= 1000, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1,2))

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv_word.fit(list(train['clear_text'].values.astype('U')) + list(test['clear_text'].values.astype('U')))
train_tfv_word =  tfv_word.transform(train['clear_text'].values.astype('U'))
test_tfv_word = tfv_word.transform(test['clear_text'].values.astype('U'))

In [14]:
# TF-IDF(char)

tfv_char = TfidfVectorizer(sublinear_tf=True,strip_accents='unicode',analyzer='char',ngram_range=(2, 6),max_features=10000)
tfv_char.fit(list(train['clear_text'].values.astype('U')) + list(test['clear_text'].values.astype('U')))
train_tfv_char = tfv_char.transform(train['clear_text'].values.astype('U'))
test_tfv_char = tfv_char.transform(test['clear_text'].values.astype('U'))

In [15]:
from scipy.sparse import hstack
# TF-IDF for test set (words + char)

train_tfidf = hstack([train_tfv_word, train_tfv_char])
test_tfidf = hstack([test_tfv_word, test_tfv_char])

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer,roc_curve, roc_auc_score

In [17]:
clf_lr = LogisticRegression() # Logistic Regression Model

print(cross_val_score(clf_lr, train_tfidf, train["type"], cv=5, scoring=make_scorer(f1_score))) # cross validation

clf_lr.fit(train_tfidf,train["type"])

preds = clf_lr.predict(test_tfidf) #predictions on test data

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.7178046  0.71560154 0.71899106 0.71716812 0.71651664]


In [18]:
print('LogReg TF-IDF', accuracy_score(test['type'], preds))

LogReg TF-IDF 0.7060923999294657
